In [1]:
from time import time as tim
import pandas as pd
import numpy as np 
from pandas import Series
domains=pd.read_csv("all_domain.csv",header=None)

import os
root_dir = os.path.abspath('../..')
task_dir= os.path.join(root_dir,'user_profile')
data_dir=os.path.join(task_dir,'data')
final_data=os.path.join(data_dir,"final_scrap")

import urllib
import re
import time
from threading import Thread
import MySQLdb
import mechanize
import readability
from bs4 import BeautifulSoup
from readability.readability import Document
import urlparse

def scraper(root,steps):
    urls = [root]
    visited = [root]
    counter = 0
    scraped=[]
    while counter < steps:
        step_url = scrapeStep(urls,scraped)
        urls = []
        for u in step_url:
            if u not in visited and str(urlparse.urlparse(u).hostname) in urlparse.urlparse(root).hostname :
                urls.append(u)
                visited.append(u)
        counter+=1
    return visited

def scrapeStep(root,scraped):
    result_urls = []
    br = mechanize.Browser()
    br.set_handle_robots(False)
    br.addheaders = [('User-agent', 'Firefox')]

    for url in root:
        if url not in scraped:
            try:
                br.open(url)
                for link in br.links():
                    newurl = urlparse.urljoin(link.base_url,link.url)
                    result_urls.append(newurl)
                scraped.append(url)
            except:
                error= "error"
    return result_urls


In [ ]:
scraps=[]
for dom,n in zip(domains[1][102:125],domains[1][102:125].index.values):
    t1=tim()
    scraps=scraps+scraper(dom,2)[1:]
    t2=tim()
    print(t2-t1)
    if (n+1)%25==0:
        Series(scraps).to_csv(os.path.join(final_data,"scraped_data_"+str(n+1)+".csv"))